In [2]:
import pandas as pd
import numpy as np

filename = '../data/raw/supply_chain_raw.csv'
pd.set_option('display.max_columns', None)

### Extract

In [3]:
raw_df = pd.read_csv(filename, encoding='cp1252')
raw_df.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,Customer Country,Customer Email,Customer Fname,Customer Id,Customer Lname,Customer Password,Customer Segment,Customer State,Customer Street,Customer Zipcode,Department Id,Department Name,Latitude,Longitude,Market,Order City,Order Country,Order Customer Id,order date (DateOrders),Order Id,Order Item Cardprod Id,Order Item Discount,Order Item Discount Rate,Order Item Id,Order Item Product Price,Order Item Profit Ratio,Order Item Quantity,Sales,Order Item Total,Order Profit Per Order,Order Region,Order State,Order Status,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Cally,20755,Holloway,XXXXXXXXX,Consumer,PR,5365 Noble Nectar Island,725.0,2,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,20755,1/31/2018 22:56,77202,1360,13.110000,0.04,180517,327.75,0.29,1,327.75,314.640015,91.250000,Southeast Asia,Java Occidental,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Irene,19492,Luna,XXXXXXXXX,Consumer,PR,2679 Rustic Loop,725.0,2,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,19492,1/13/2018 12:27,75939,1360,16.389999,0.05,179254,327.75,-0.80,1,327.75,311.359985,-249.089996,South Asia,Rajastán,PENDING,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,EE. UU.,XXXXXXXXX,Gillian,19491,Maldonado,XXXXXXXXX,Consumer,CA,8510 Round Bear Gate,95125.0,2,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,19491,1/13/2018 12:06,75938,1360,18.030001,0.06,179253,327.75,-0.80,1,327.75,309.720001,-247.779999,South Asia,Rajastán,CLOSED,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,EE. UU.,XXXXXXXXX,Tana,19490,Tate,XXXXXXXXX,Home Office,CA,3200 Amber Bend,90027.0,2,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,19490,1/13/2018 11:45,75937,1360,22.940001,0.07,179252,327.75,0.08,1,327.75,304.809998,22.860001,Oceania,Queensland,COMPLETE,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,Puerto Rico,XXXXXXXXX,Orli,19489,Hendricks,XXXXXXXXX,Corporate,PR,8671 Iron Anchor Corners,725.0,2,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,19489,1/13/2018 11:24,75936,1360,29.500000,0.09,179251,327.75,0.45,1,327.75,298.250000,134.210007,Oceania,Queensland,PENDING_PAYMENT,NaN,1360,73,NaN,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


In [4]:
raw_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 180519 entries, 0 to 180518
Data columns (total 53 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Type                           180519 non-null  str    
 1   Days for shipping (real)       180519 non-null  int64  
 2   Days for shipment (scheduled)  180519 non-null  int64  
 3   Benefit per order              180519 non-null  float64
 4   Sales per customer             180519 non-null  float64
 5   Delivery Status                180519 non-null  str    
 6   Late_delivery_risk             180519 non-null  int64  
 7   Category Id                    180519 non-null  int64  
 8   Category Name                  180519 non-null  str    
 9   Customer City                  180519 non-null  str    
 10  Customer Country               180519 non-null  str    
 11  Customer Email                 180519 non-null  str    
 12  Customer Fname                 180519 non

In [5]:
for col in raw_df.select_dtypes(include=['object', 'string']).columns: 
    
    mask_cortos = raw_df[col].str.strip().str.len() <= 10
    
    # Solo mostramos si realmente encontramos algo
    if mask_cortos.any():
        print(raw_df.loc[mask_cortos, col].value_counts().head(5))
        print("-" * 30)

Type
DEBIT       69295
TRANSFER    49883
PAYMENT     41725
CASH        19616
Name: count, dtype: int64
------------------------------
Category Name
Cleats        24551
Fishing       17325
Golf Balls     1475
Trade-In        974
Hockey          614
Name: count, dtype: int64
------------------------------
Customer City
Caguas      66770
Chicago      3885
Brooklyn     3412
New York     1816
Bronx        1500
Name: count, dtype: int64
------------------------------
Customer Country
EE. UU.    111146
Name: count, dtype: int64
------------------------------
Customer Email
XXXXXXXXX    180519
Name: count, dtype: int64
------------------------------
Customer Fname
Mary       65150
James       1835
Robert      1759
Michael     1680
David       1625
Name: count, dtype: int64
------------------------------
Customer Lname
Smith       64104
Johnson       989
Brown         909
Williams      869
Jones         859
Name: count, dtype: int64
------------------------------
Customer Password
XXXXXXXXX    

In [6]:
clean_df = raw_df.copy()

### Transform

### 2. Estructura

In [7]:
clean_df.columns = clean_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace(r'\(|\)','',regex=True)
clean_df = clean_df[clean_df.columns.difference(['product_description', 'customer_email', 'customer_password', 'product_image', 'order_zipcode', 'product_status'])]

### 3. Limpieza basica de texto

In [8]:
for col in clean_df.select_dtypes(include=['object', 'string']).columns:
    clean_df[col] = clean_df[col].str.strip().str.title()

for col in ['customer_state', 'type', 'order_status', 'market']:
    clean_df[col] = clean_df[col].str.upper()

fix_dict = {
    'Ee. Uu.': 'EE. UU.'
}

clean_df['customer_country'] = clean_df['customer_country'].replace(fix_dict)

### 4. Conversion de tipos

In [9]:
clean_df['customer_zipcode'] = clean_df['customer_zipcode'].astype('Int64')
clean_df['late_delivery_risk'] = clean_df['late_delivery_risk'].astype('boolean')

clean_df['order_date_dateorders'] = pd.to_datetime(clean_df['order_date_dateorders'], format='%m/%d/%Y %H:%M', errors='coerce')
if clean_df['order_date_dateorders'].isnull().sum() > 0:
    print("Se generaron fechas invalidas! - order_date_dateorders")
else:
    print("Todas las fechas se convirtieron correctamente - order_date_dateorders")
    
clean_df['order_status'] = clean_df['order_status'].astype('category')

clean_df['shipping_date_dateorders'] = pd.to_datetime(clean_df['shipping_date_dateorders'], format='%m/%d/%Y %H:%M', errors='coerce')
if clean_df['shipping_date_dateorders'].isnull().sum() > 0:
    print("Se generaron fechas invalidas! - shipping_date_dateorders")
else:
    print("Todas las fechas se convirtieron correctamente - shipping_date_dateorders")
    
clean_df['shipping_mode'] = clean_df['shipping_mode'].astype('category')
clean_df['type'] = clean_df['type'].astype('category')


Todas las fechas se convirtieron correctamente - order_date_dateorders
Todas las fechas se convirtieron correctamente - shipping_date_dateorders


In [39]:
for col in clean_df.select_dtypes(include=['float64']).columns.difference(['latitude','longitude']):
    clean_df[col] = clean_df[col].round(2)

### 5. Duplicados

In [10]:
clean_df = clean_df.drop_duplicates()
clean_df = clean_df.drop_duplicates(subset=['order_id', 'order_customer_id', 'order_item_id', 'order_date_dateorders'],keep='first')

### 6. Nulos

In [11]:
clean_df = clean_df.dropna(subset=['customer_zipcode'])
clean_df = clean_df.dropna(subset=['category_id', 'customer_id', 'department_id', 'order_customer_id', 'order_id', 'order_item_cardprod_id', 'order_item_id', 'product_card_id', 'product_category_id'], how='any')


### 7. Rangos

In [17]:
date_condition = clean_df['shipping_date_dateorders'] < clean_df['order_date_dateorders']
clean_df[date_condition]

,benefit_per_order,category_id,category_name,customer_city,customer_country,customer_fname,customer_id,customer_lname,customer_segment,customer_state,customer_street,customer_zipcode,days_for_shipment_scheduled,days_for_shipping_real,delivery_status,department_id,department_name,late_delivery_risk,latitude,longitude,market,order_city,order_country,order_customer_id,order_date_dateorders,order_id,order_item_cardprod_id,order_item_discount,order_item_discount_rate,order_item_id,order_item_product_price,order_item_profit_ratio,order_item_quantity,order_item_total,order_profit_per_order,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,sales,sales_per_customer,shipping_date_dateorders,shipping_mode,type


In [18]:
quantity_condition = clean_df['order_item_quantity'] < 0
clean_df[quantity_condition]

,benefit_per_order,category_id,category_name,customer_city,customer_country,customer_fname,customer_id,customer_lname,customer_segment,customer_state,customer_street,customer_zipcode,days_for_shipment_scheduled,days_for_shipping_real,delivery_status,department_id,department_name,late_delivery_risk,latitude,longitude,market,order_city,order_country,order_customer_id,order_date_dateorders,order_id,order_item_cardprod_id,order_item_discount,order_item_discount_rate,order_item_id,order_item_product_price,order_item_profit_ratio,order_item_quantity,order_item_total,order_profit_per_order,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,sales,sales_per_customer,shipping_date_dateorders,shipping_mode,type


In [19]:
for col in clean_df.select_dtypes(include=['int64','Int64','float64']).columns:
    if (clean_df[col] < 0).any():
        print(col)

benefit_per_order
latitude
longitude
order_item_profit_ratio
order_profit_per_order


In [47]:
clean_df.loc[clean_df['order_item_quantity'] > 1, :].head()

,benefit_per_order,category_id,category_name,customer_city,customer_country,customer_fname,customer_id,customer_lname,customer_segment,customer_state,customer_street,customer_zipcode,days_for_shipment_scheduled,days_for_shipping_real,delivery_status,department_id,department_name,late_delivery_risk,latitude,longitude,market,order_city,order_country,order_customer_id,order_date_dateorders,order_id,order_item_cardprod_id,order_item_discount,order_item_discount_rate,order_item_id,order_item_product_price,order_item_profit_ratio,order_item_quantity,order_item_total,order_profit_per_order,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,sales,sales_per_customer,shipping_date_dateorders,shipping_mode,type
48,-30.75,17,Cleats,Bayamon,Puerto Rico,Mary,9083,Frank,Home Office,PR,75 Sunny Grounds,957,2,5,Late Delivery,4,Apparel,True,18.380119,-66.183128,PACIFIC ASIA,Mirzapur,India,9083,2016-02-24 13:57:00,28744,365,4.8,0.04,71956,59.99,-0.27,2,115.18,-30.75,South Asia,Uttar Pradesh,PENDING_PAYMENT,365,17,Perfect Fitness Perfect Rip Deck,59.99,119.98,115.18,2016-02-29 13:57:00,Second Class,PAYMENT
49,-122.73,29,Shop By Sport,Caguas,Puerto Rico,Mary,4741,Smith,Home Office,PR,9731 Honey Fox Towers,725,2,2,Shipping On Time,5,Golf,False,18.235573,-66.370613,PACIFIC ASIA,Bursa,Turquía,4741,2016-10-25 14:39:00,45461,627,0.8,0.01,113598,39.99,-1.55,2,79.18,-122.73,West Asia,Bursa,PENDING_PAYMENT,627,29,Under Armour Girls' Toddler Spine Surge Runni,39.99,79.98,79.18,2016-10-27 14:39:00,Second Class,PAYMENT
50,33.60,24,Women'S Apparel,Caguas,Puerto Rico,Elizabeth,639,Pittman,Home Office,PR,7573 Golden Treasure Centre,725,2,6,Late Delivery,5,Golf,True,18.025368,-66.613037,PACIFIC ASIA,Murray Bridge,Australia,639,2016-03-30 04:37:00,31115,502,4.0,0.04,77757,50.00,0.35,2,96.00,33.60,Oceania,Australia Del Sur,PENDING_PAYMENT,502,24,Nike Men'S Dri-Fit Victory Golf Polo,50.00,100.00,96.00,2016-04-05 04:37:00,Second Class,PAYMENT
51,24.69,29,Shop By Sport,Caguas,Puerto Rico,Katherine,9702,Tyler,Home Office,PR,8369 Sunny Crossing,725,2,2,Shipping On Time,5,Golf,False,18.273838,-66.370636,PACIFIC ASIA,Kartal,Turquía,9702,2016-10-30 01:31:00,45766,627,4.0,0.05,114401,39.99,0.33,2,75.98,24.69,West Asia,Estambul,PENDING_PAYMENT,627,29,Under Armour Girls' Toddler Spine Surge Runni,39.99,79.98,75.98,2016-11-01 01:31:00,Second Class,PAYMENT
52,9.10,24,Women'S Apparel,Caguas,Puerto Rico,Mary,9114,Smith,Home Office,PR,1425 Fallen Fox Arbor,725,2,3,Late Delivery,5,Golf,True,18.284805,-66.370590,PACIFIC ASIA,Ulan Bator,Mongolia,9114,2016-11-28 01:18:00,47752,502,9.0,0.09,119405,50.00,0.10,2,91.00,9.10,Eastern Asia,Ulán Bator,PENDING_PAYMENT,502,24,Nike Men'S Dri-Fit Victory Golf Polo,50.00,100.00,91.00,2016-12-01 01:18:00,Second Class,PAYMENT


In [49]:
condition = ((clean_df['order_item_discount_rate']* clean_df['order_item_quantity']) * clean_df['order_item_product_price']).round(2) != clean_df['order_item_discount']
clean_df.loc[condition, ['order_item_discount_rate','order_item_product_price','order_item_discount','order_item_quantity']]

,order_item_discount_rate,order_item_product_price,order_item_discount,order_item_quantity
2,0.06,327.75,18.03,1
20,0.06,327.75,18.03,1
38,0.06,327.75,18.03,1
77,0.06,59.99,9.90,3
94,0.06,39.99,11.00,5
...,...,...,...,...
180430,0.06,399.98,22.00,1
180448,0.06,399.98,22.00,1
180466,0.06,399.98,22.00,1
180484,0.06,399.98,22.00,1


In [57]:
condition = ((clean_df['order_item_product_price'] * clean_df['order_item_quantity']).round(2) - clean_df['order_item_discount']).round(2) != clean_df['order_item_total']
clean_df.loc[condition, ['order_item_product_price', 'order_item_quantity', 'order_item_discount', 'order_item_total']]

,order_item_product_price,order_item_quantity,order_item_discount,order_item_total
5,327.75,1,32.78,294.98
11,327.75,1,59.00,268.76
16,327.75,1,6.56,321.20
23,327.75,1,32.78,294.98
29,327.75,1,59.00,268.76
...,...,...,...,...
180441,399.98,1,100.00,299.99
180459,399.98,1,100.00,299.99
180477,399.98,1,100.00,299.99
180495,399.98,1,100.00,299.99


In [61]:
calculo_total = (clean_df['order_item_product_price'] * clean_df['order_item_quantity']) - clean_df['order_item_discount']
condition = ~np.isclose(clean_df['order_item_total'], calculo_total, atol=0.02)

clean_df[condition]

,benefit_per_order,category_id,category_name,customer_city,customer_country,customer_fname,customer_id,customer_lname,customer_segment,customer_state,customer_street,customer_zipcode,days_for_shipment_scheduled,days_for_shipping_real,delivery_status,department_id,department_name,late_delivery_risk,latitude,longitude,market,order_city,order_country,order_customer_id,order_date_dateorders,order_id,order_item_cardprod_id,order_item_discount,order_item_discount_rate,order_item_id,order_item_product_price,order_item_profit_ratio,order_item_quantity,order_item_total,order_profit_per_order,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,sales,sales_per_customer,shipping_date_dateorders,shipping_mode,type
